In [391]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

# seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
# random.seed(seed)
# torch.manual_seed(seed)
# print(seed)
# random.seed(1320387042447901345)
# torch.manual_seed(1320387042447901345)
tm = TsetlinMachine(train_x.shape[1], 5)
for i in range(6):
    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    for x,y in zip(shuffled_x, shuffled_y):
        out_1 = tm.forward(x.unsqueeze(0))
        tm.update(y.unsqueeze(0))
        out_2 = tm.forward(x.unsqueeze(0))
        assert torch.equal(y.unsqueeze(0),out_2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [392]:
tm.l1.W,tm.l2.W[:, :tm.l2.W.shape[1]//2],tm.l2.W[:, tm.l2.W.shape[1]//2:]

(tensor([[ 0,  0, 37, 37, 37,  0],
         [ 0, 37, 37, 32,  0,  0],
         [ 0,  0,  0, 16,  0, 16],
         [37,  0,  0,  0, 32,  0],
         [ 0,  0,  0,  0,  0, 22]]),
 tensor([[0, 0, 0, 0, 0]]),
 tensor([[31, 31, 15,  0,  0]]))

In [393]:
out = tm.forward(train_x)
assert torch.equal(train_y, out)

In [394]:
failed_count = 0
for _ in range(1000):
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(6):
        shuffled_idx = torch.randperm(train_x.shape[0])
        shuffled_x = train_x[shuffled_idx]
        shuffled_y = train_y[shuffled_idx]

        for j, (x, y) in enumerate(zip(shuffled_x, shuffled_y)):
            out_1 = tm.forward(x.unsqueeze(0))
            tm.update(y.unsqueeze(0))
            out_2 = tm.forward(x.unsqueeze(0))
            assert torch.equal(y.unsqueeze(0), out_2)

    out = tm.forward(train_x)
    if not torch.equal(train_y, out):
        failed_count += 1

failed_count

187

In [397]:
tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(train_x)
out_1, train_y

(tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 1, 0, 0, 0, 0]))